In [1]:
#Import libraries
import numpy as np
import os
import pickle as pickle
from itertools import product, permutations
from cymetric.pointgen.pointgen import PointGenerator
from cymetric.models.tfhelper import prepare_tf_basis
import tensorflow as tf
tfk = tf.keras

In [2]:
#Set up point generator
n_coords = 5
monomial_powers = 5
monomials = monomial_powers*np.eye(n_coords, dtype=np.int64)
coefficients = np.ones(n_coords)
kmoduli = np.ones(1)
ambient = np.array([n_coords-1])
#print(monomials,coefficients,kmoduli,ambient)
pg = PointGenerator(monomials, coefficients, kmoduli, ambient) ###...better to import this directly somehow?

#Generate points
n_pts = 1000
dirname = 'cy_models/link_data'
kappa = pg.prepare_dataset(n_pts, dirname, val_split=0.)
data = np.load(os.path.join(dirname, 'dataset.npz'))
pg.prepare_basis(dirname, kappa=kappa)


/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
pointgen:INFO:Vol_k: 5.0, Vol_cy: 287.45090973298755.


0

In [3]:
#Import the pre-trained CYMetric model
#Generate data as cymetric processable objects
from cymetric.models.tfhelper import prepare_tf_basis
dirname = 'cy_models/link_data'
data = np.load(os.path.join(dirname, 'dataset.npz'))
points = data['X_train']
BASIS = np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True)
BASIS = prepare_tf_basis(BASIS)

#Reinstate model structure
n_in = 2*5
n_out = 1
nlayer = 3
nHidden = 64
act = 'gelu'
alpha = [1., 1., 1., 1., 1.]
nn_phi = tfk.Sequential()
nn_phi.add(tfk.Input(shape=(n_in)))
for i in range(nlayer):
    nn_phi.add(tfk.layers.Dense(nHidden, activation=act))
nn_phi.add(tfk.layers.Dense(n_out, use_bias=False))

#Import the model
from cymetric.models.tfmodels import PhiFSModel
cymodel_filepath = os.path.dirname(os.getcwd())+'/models/cy_models/cy_metric_model.keras'
cymetric_model = PhiFSModel(nn_phi, BASIS, alpha=alpha)
cymetric_model.nn_phi = tfk.models.load_model(cymodel_filepath)
#cymetric_model.summary()


In [4]:
#Define necessary functions
def CoordChange_C5R10(points, inverse=False):
    """
    Vectorized coordinate change between complex and real representations. 
    Real coordinates arranged: (x1, x2, ..., y1, y2, ...).
    
    Args:
        points (np.ndarray): 
            - If `inverse=False`: array of shape (batch_size, n_complex), dtype complex.
            - If `inverse=True`: array of shape (batch_size, 2 * n_complex), dtype float.
        inverse (bool): Direction of transformation.

    Returns:
        np.ndarray:
            - If `inverse=False`: shape (batch_size, 2 * n_complex), dtype float.
            - If `inverse=True`: shape (batch_size, n_complex), dtype complex.
    """
    if not inverse:
        #return np.stack((point.real, point.imag), axis=-1).reshape(points.shape[0], -1) #...form of (x1, y1, x2, y2, ...)
        return np.concatenate((np.real(points), np.imag(points)), axis=1)
    else:
        #return point[:,::2] + 1j * point[:,1::2] #...form of (x1, y1, x2, y2, ...)
        return points[:, :points.shape[1] // 2] + 1j * points[:, points.shape[1] // 2:]


def hermitian_to_real_symmetric(H_batch):
    """
    Vectorized conversion of a batch of 3x3 Hermitian matrices to 6x6 real symmetric matrices.
    Real coordinates arranged: (x1, x2, ..., y1, y2, ...).
    
    Args:
        H_batch (np.ndarray): Array of shape (batch_size, 3, 3) with complex Hermitian matrices.

    Returns:
        G_batch (np.ndarray): Array of shape (batch_size, 6, 6) with real symmetric matrices.
    """
    if H_batch.shape[-2:] != (3, 3):
        raise ValueError("Each matrix must be 3x3.")
    
    if not np.allclose(H_batch, H_batch.conj().transpose(0, 2, 1)):
        raise ValueError("All input matrices must be Hermitian.")

    A = H_batch.real  # shape: (batch_size, 3, 3)
    B = H_batch.imag  # shape: (batch_size, 3, 3)

    upper = np.concatenate((A, -B), axis=2)  # shape: (batch_size, 3, 6)
    lower = np.concatenate((B,  A), axis=2)  # shape: (batch_size, 3, 6)
    G_batch = np.concatenate((upper, lower), axis=1)  # shape: (batch_size, 6, 6)

    # Reorder axes: (x1, y1, x2, y2, x3, y3) → (x1, x2, x3, y1, y2, y3)
    perm = [0, 2, 4, 1, 3, 5]
    G_batch = G_batch[:, perm][:, :, perm]

    return G_batch


def kahler_form_real(H_batch):
    """
    Vectorized Kähler form computation for a batch of 3x3 Hermitian matrices.
    Real coordinates arranged: (x1, x2, ..., y1, y2, ...).
    
    Args:
        H_batch (np.ndarray): Array of shape (batch_size, 3, 3), complex Hermitian matrices.

    Returns:
        omega_batch (np.ndarray): Array of shape (batch_size, 6, 6), real antisymmetric matrices.
    """
    if H_batch.shape[-2:] != (3, 3):
        raise ValueError("Each input matrix must be 3x3.")

    if not np.allclose(H_batch, H_batch.conj().transpose(0, 2, 1)):
        raise ValueError("All matrices must be Hermitian.")

    A = H_batch.real  # shape: (batch_size, 3, 3)
    zero = np.zeros_like(A)

    # Assemble block antisymmetric matrix:
    # [[ 0,  A ],
    #  [ -A.T, 0 ]]
    upper = np.concatenate((zero, A), axis=2)          # shape: (batch_size, 3, 6)
    lower = np.concatenate((-A.transpose(0, 2, 1), zero), axis=2)  # shape: (batch_size, 3, 6)
    omega = np.concatenate((upper, lower), axis=1)     # shape: (batch_size, 6, 6)

    # Reorder (x1, y1, x2, y2, x3, y3) → (x1, x2, x3, y1, y2, y3)
    perm = [0, 2, 4, 1, 3, 5]
    omega_reordered = omega[:, perm][:, :, perm]

    return omega_reordered


def wedge_form2_with_form1(omega6_batch, alpha7_batch):
    """
    Vectorized wedge product of a batch of 6x6 antisymmetric 2-forms with a batch of 7D 1-forms.
    
    Args:
        omega6_batch (np.ndarray): shape (batch_size, 6, 6), antisymmetric 2-forms.
        alpha7_batch (np.ndarray): shape (batch_size, 7), 1-forms.

    Returns:
        form3_batch (np.ndarray): shape (batch_size, 7, 7, 7), antisymmetric 3-forms.
    """
    batch_size = omega6_batch.shape[0]
    
    assert omega6_batch.shape == (batch_size, 6, 6), "omega6 must be (N, 6, 6)"
    assert alpha7_batch.shape == (batch_size, 7), "alpha7 must be (N, 7)"

    # Check antisymmetry
    if not np.allclose(omega6_batch, -omega6_batch.transpose(0, 2, 1)):
        raise ValueError("omega6 must be antisymmetric")

    # Extend omega6 to (N, 7, 7)
    omega7_batch = np.zeros((batch_size, 7, 7), dtype=omega6_batch.dtype)
    omega7_batch[:, :6, :6] = omega6_batch

    # Broadcast and compute the 3-form wedge product
    i, j, k = np.meshgrid(np.arange(7), np.arange(7), np.arange(7), indexing='ij')

    # Compute each term: omega[i,j]*alpha[k], omega[j,k]*alpha[i], omega[k,i]*alpha[j]
    term1 = omega7_batch[:, i, j] * alpha7_batch[:, k]
    term2 = omega7_batch[:, j, k] * alpha7_batch[:, i]
    term3 = omega7_batch[:, k, i] * alpha7_batch[:, j]

    form3_batch = (1.0 / 3.0) * (term1 + term2 + term3) #...only add the cyclical components (acyclical are the same contribution so would double this value which then cancels with the 2 in the rank! factro to leave division by 3 instead of 6(=3!)

    return form3_batch


def holomorphic_volume_form_to_real_tensor(c_batch):
    """
    Convert a complex coefficient c of dz^1 ^ dz^2 ^ dz^3 into a real 6x6x6 tensor 
    representing the real coordinate expression of the holomorphic volume form.

    Args:
        c_batch (np.ndarray): shape (batch,), complex dtype.

    Returns:
        Omega_real (np.ndarray): shape (batch, 6, 6, 6), real part of 3-form
        Omega_imag (np.ndarray): shape (batch, 6, 6, 6), imaginary part of 3-form
    """
    batch_size = c_batch.shape[0]
    Omega_real = np.zeros((batch_size, 6, 6, 6))
    Omega_imag = np.zeros((batch_size, 6, 6, 6))

    dx = [0, 1, 2]
    dy = [3, 4, 5]
    index_triplets = list(permutations(dx, 3))

    # Only one unique permutation set (since all permutations are included later)
    for (i, j, k) in index_triplets:
        iy, jy, ky = dy[i], dy[j], dy[k]
        for a, b, d in product([0, 1], repeat=3):
            idx = [i, j, k]
            idx[0] = [i, iy][a]
            idx[1] = [j, jy][b]
            idx[2] = [k, ky][d]

            coeff = ((1j) ** (a + b + d)) * c_batch[:, None] / 6  # shape: (batch, 1)
            for perm in set(permutations(idx)):
                # Determine the sign of the permutation
                sign = (
                    1 if list(perm) == sorted(perm) else
                    (-1) ** sum(p1 > p2 for p1, p2 in zip(perm, sorted(perm)))
                )
                Omega_real[:, perm[0], perm[1], perm[2]] += coeff.real[:, 0] * sign
                Omega_imag[:, perm[0], perm[1], perm[2]] += coeff.imag[:, 0] * sign

    return Omega_real, Omega_imag


In [5]:
#Define the class for the link points
class LinkPoints:
    def __init__(self, cy_points):
        # Setup the input point information
        self.cy_points = cy_points
        self.one_idxs = np.argmax(np.isclose(self.cy_points, complex(1, 0)), axis=1) #...find the index which is set to 1 to define the patch
        self.dropped_idxs = pg._find_max_dQ_coords(self.cy_points)

        # Generate the link point data
        self.cy_points_in_S9 = self.cy_points/np.linalg.norm(self.cy_points, axis=1).reshape(-1, 1)
        self.thetas = np.random.uniform(low=0., high=2*np.pi, size=self.cy_points.shape[0]) #...sample a random angle
        self.link_points = self.cy_points_in_S9 * np.exp(1j * self.thetas.reshape(-1, 1)) #...multiple that angle in, the plane origin is the normalised cy point and orientation is implicitly set
        self.dthetas = self.cy_points_in_S9 * 1j * np.exp(1j * self.thetas.reshape(-1, 1)) #...the position vector 90degrees around the S1 is parallel to this d\theta, and cause its on sphere it is normalised already
        #self.dtheta_R1 = ### --> this is just self.theta?

        # Transform to the link coordinate system
        mask = np.ones(self.link_points.shape, dtype=bool)
        rows = np.arange(self.link_points.shape[0])
        mask[rows, self.one_idxs] = False
        mask[rows, self.dropped_idxs] = False
        #c3_coords = self.link_points[mask].reshape(self.link_points.shape[0], -1)
        c3_coords = self.cy_points[mask].reshape(self.cy_points.shape[0], -1)
        self.link_points_local = np.concatenate((np.real(c3_coords), np.imag(c3_coords), self.thetas.reshape(-1, 1)), axis=1) #...generate the link local coordinates
        
        # Define the geometric components
        self.holomorphic_volume_form = pg.holomorphic_volume_form(self.cy_points) # (note pg is external)
        self.hvf_r, self.hvf_i = holomorphic_volume_form_to_real_tensor(self.holomorphic_volume_form)
        self.hermitian_metric = cymetric_model(CoordChange_C5R10(self.cy_points)).numpy()
        self.riemannian_metric_R6 = hermitian_to_real_symmetric(self.hermitian_metric)
        self.kahler_form_R6 = kahler_form_real(self.hermitian_metric)
        self.phi_R7 = wedge_form2_with_form1(self.kahler_form_R6, np.concatenate((np.zeros((self.cy_points.shape[0], 6)), self.thetas.reshape(-1, 1)), axis=1))
        self.phi_R7[:, :6, :6, :6] += self.hvf_i

        pass

Link_Dataset = LinkPoints(CoordChange_C5R10(points,inverse=True))


In [6]:
#Print example points
pt_idx = 200

print(f'Example point:\n{np.round(Link_Dataset.link_points_local[pt_idx], 3)}\n')
print(f'Respective 3-form:\n{np.round(Link_Dataset.phi_R7[pt_idx], 3)}')


Example point:
[-0.533  0.739 -0.06  -0.783 -0.556  0.272  0.585]

Respective 3-form:
[[[ 0.     0.     0.     0.     0.     0.     0.   ]
  [ 0.     0.    -0.17   0.     0.    -0.156  0.   ]
  [ 0.     0.17   0.     0.    -0.156  0.     0.008]
  [ 0.     0.     0.     0.     0.     0.     0.   ]
  [ 0.     0.     0.156  0.     0.     0.17   0.017]
  [ 0.     0.156  0.     0.    -0.17   0.     0.001]
  [ 0.     0.    -0.008  0.    -0.017 -0.001  0.   ]]

 [[ 0.     0.     0.17   0.     0.     0.156  0.   ]
  [ 0.     0.     0.     0.     0.     0.     0.   ]
  [-0.17   0.     0.    -0.156  0.     0.     0.001]
  [ 0.     0.     0.156  0.     0.     0.17   0.   ]
  [ 0.     0.     0.     0.     0.     0.     0.001]
  [-0.156  0.     0.    -0.17   0.     0.     0.018]
  [ 0.     0.    -0.001  0.    -0.001 -0.018  0.   ]]

 [[ 0.     0.17   0.     0.     0.156  0.    -0.008]
  [ 0.17   0.     0.     0.156  0.     0.    -0.001]
  [ 0.     0.     0.     0.     0.     0.     0.   ]
  [ 0.   

In [7]:
###testingggg
from ..geometry.wedge_product import wedge_product
test_pt_idx = 21
Link_Dataset.kahler_form_R6.shape
#
#wedge_form2_with_form1(Link_Dataset.kahler_form_R6, np.concatenate((np.zeros((self.cy_points.shape[0], 6)), self.thetas.reshape(-1, 1)), axis=1))


ImportError: attempted relative import with no known parent package